In [1]:
### 27号我有一波新的训练数据，看看成色。
import os, json
import re

In [2]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/algebra/hcot_math_algebra_0124.jsonl",
    "r",
) as f:
    hcot_math_algebra_new = f.readlines()

with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/counting_and_probability/hcot_math_counting_and_probability_0124.jsonl",
    "r",
) as f:
    hcot_math_counting_new = f.readlines()

with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/number_theory/hcot_math_number_theory_0124.jsonl",
    "r",
) as f:
    hcot_math_number_theory_new = f.readlines()

with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/prealgebra/hcot_math_prealgebra_0124.jsonl",
    "r",
) as f:
    hcot_math_prealgebra_new = f.readlines()

In [3]:
hcot_math_new = (
    hcot_math_algebra_new
    + hcot_math_counting_new
    + hcot_math_number_theory_new
    + hcot_math_prealgebra_new
)

In [4]:
def fix_incomplete_tags_v2(text: str) -> str:
    tags = ["thought", "equation"]

    for tag in tags:
        open_tag = f"<{tag}>"
        close_tag = f"</{tag}>"
        incomplete_situation_open_tag_left = f"<{tag}"
        incomplete_situation_open_tag_right = f"{tag}>"
        incomplete_situation_close_tag_left = f"/{tag}>"
        incomplete_situation_close_tag_right = f"</{tag}"
        text_tmp = text.replace(open_tag, "&*#$%")
        text_tmp = text_tmp.replace(close_tag, "@#!$%")
        text_tmp = text_tmp.replace(incomplete_situation_close_tag_left, close_tag)
        text_tmp = text_tmp.replace(close_tag, "@#!$%")

        text_tmp = text_tmp.replace(incomplete_situation_close_tag_right, close_tag)
        text_tmp = text_tmp.replace(close_tag, "@#!$%")

        text_tmp = text_tmp.replace(incomplete_situation_open_tag_left, open_tag)
        text_tmp = text_tmp.replace(open_tag, "&*#$%")
        text_tmp = text_tmp.replace(incomplete_situation_open_tag_right, open_tag)
        text_tmp = text_tmp.replace(open_tag, "&*#$%")

        text_tmp = text_tmp.replace("&*#$%", open_tag)
        text_tmp = text_tmp.replace("@#!$%", close_tag)
        text = text_tmp
    return text


# Example usage
example_string = "This is a test equation>xample without proper tag. Here is another \n/thought> with no end."
fixed_string = fix_incomplete_tags_v2(example_string)
fixed_string

'This is a test <equation>xample without proper tag. Here is another \n</thought> with no end.'

In [5]:
new_hcot_math_new = []
count_fix = []
for line in hcot_math_new:
    line_js = json.loads(line)
    one_response = line_js["response"]
    one_new_response = one_response.replace("\n\n", "\n")
    line_js["response"] = one_new_response
    if line_js["response"]:
        response_origin = line_js["response"]
        response_fix = fix_incomplete_tags_v2(response_origin)
        if response_fix != response_origin:
            count_fix.append((response_fix, response_origin))
        line_js["response"] = response_fix
        new_hcot_math_new.append(line_js)

In [9]:
with open("./data/MATH/hcot_math_fix_0126.jsonl", "w") as f:
    for line in new_hcot_math_new:
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

### 构建带COT的直接训练数据

In [10]:
import os, json
from tqdm import tqdm

with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/hcot_math_fix_0126.jsonl",
    "r",
) as f:
    h_cot_math = f.readlines()


def build_direct_conversations(one_gsm8k_input):
    one_gsm8k_input["response"] = one_gsm8k_input["response"].replace(
        "[assistant'](#math_action)", "[assistant](#math_action)"
    )
    model_response = (
        one_gsm8k_input["response"]
        .split("[assistant](#math_action)")[1]
        .strip("\\n")
        .replace("\\n", "\n")
    )
    one_gsm8k_input["conversations"] = [
        {"from": "human", "value": one_gsm8k_input["meta"]["question"]},
        {"from": "gpt", "value": model_response},
    ]
    return one_gsm8k_input


h_cot_math_direct = []
for one_gsm8k in h_cot_math:
    one_gsm8k_js = json.loads(one_gsm8k)
    if one_gsm8k_js["response"]:
        h_cot_math_direct.append(build_direct_conversations(one_gsm8k_js))

In [12]:
len(h_cot_math_direct)

4589

In [13]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/h_cot_direct_conv_0126.json",
    "w",
) as f:
    json.dump(h_cot_math_direct, f, ensure_ascii=False)

#### 构建hcot的cot model的训练数据

In [14]:
import re


def remove_thoughts(text):
    """
    Removes all occurrences of <thought>xxx</thought> from the text.

    Args:
    text (str): The input string containing <thought>xxx</thought> tags.

    Returns:
    str: The modified string with <thought>xxx</thought> tags removed.
    """
    # Pattern to match <thought>xxx</thought> and optional preceding and following newlines
    pattern = r"(?<=\n)<thought>.*?</thought>(?=\n)"

    # Replace all occurrences with an empty string
    modified_text = re.sub(pattern, "<[COT]>", text, flags=re.DOTALL)

    return modified_text


def extract_thoughts_accumulative(problem, text):
    pairs = []
    previous_end = 0
    # prefix = f"Solve the following math word problem with your step-by-step analysis, {problem}. I will provide the previous analysis and equations, your job is to show me your thought of the next action, you do not need to give the calculation equation.\n\n"
    prefix = f"{problem}"

    while "<thought>" in text[previous_end:]:
        # Find the next <thought> tag
        start_thought = text.find("<thought>", previous_end)
        end_thought = text.find("</thought>", start_thought)

        # Break the loop if no more <thought> tag is found
        if start_thought == -1 or end_thought == -1:
            break

        # Extract the input and output
        input_text = text[:start_thought].strip()
        output_text = text[
            start_thought + 9 : end_thought
        ].strip()  # 9 is the length of '<thought>'

        # Append the pair to the result
        input_text_tmp = prefix + remove_thoughts(input_text)

        pairs.append(
            {
                "conversations": [
                    {"from": "human", "value": input_text_tmp + "\n<[COT]>"},
                    {"from": "gpt", "value": output_text},
                ],
                "Query": input_text_tmp,
                "COT_target": "\n<[COT]>",
                "thought_target": output_text,
            }
        )

        # Update the end position for the next iteration
        previous_end = end_thought + 10

    return pairs

In [15]:
cot_model_train_data = []
for one_gsm8k_direct in h_cot_math_direct:
    new_train_data = extract_thoughts_accumulative(
        one_gsm8k_direct["conversations"][0]["value"],
        one_gsm8k_direct["conversations"][1]["value"],
    )
    cot_model_train_data.extend(new_train_data)

In [20]:
cot_model_train_data[-10087]

{'conversations': [{'from': 'human',
   'value': "Find the sum of the squares of the solutions of $x^2-13x+4=0$.The problem is asking us to find the sum of squares of solutions (roots) of a quadratic equation. We can use Vieta's formulas, which relate coefficients in a polynomial to sums and products of its roots.\n<[COT]>"},
  {'from': 'gpt',
   'value': "Firstly, let's denote $r_1$ and $r_2$ as roots of given polynomial. According to Vieta's formulas, we know that $r_1 + r_2 = 13$, which is equal to negative coefficient at x divided by coefficient at $x^2$. Also, product of roots equals constant term divided by coefficient at $x^2$, so we have that $r_1 \\cdot r_2 = 4$.</[COT]>"}],
 'Query': "Find the sum of the squares of the solutions of $x^2-13x+4=0$.The problem is asking us to find the sum of squares of solutions (roots) of a quadratic equation. We can use Vieta's formulas, which relate coefficients in a polynomial to sums and products of its roots.",
 'COT_target': '\n<[COT]>',


In [19]:
for d in cot_model_train_data:
    d["conversations"][1]["value"] = d["conversations"][1]["value"] + "</[COT]>"

In [21]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/COT_model_train_data_kun_idea_0126.json",
    "w",
) as f:
    json.dump(cot_model_train_data, f, ensure_ascii=False)

In [34]:
cot_model_train_data[0]["conversations"]

[{'from': 'human',
  'value': "Factor $t^2-121$.The problem is asking us to factorize a quadratic expression, $t^2 - 121$. This can be done by recognizing that it's a difference of squares, which can be factored into two binomial expressions.\n<[COT]>"},
 {'from': 'gpt',
  'value': 'The given expression $t^2 - 121$ is in the form of $a^2 - b^2$, where $a = t$ and $b = 11$. The difference of squares formula states that any expression in this form can be factored as $(a-b)(a+b)$.</[COT]>'}]

In [35]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/COT_model_train_data_kun_idea_0126.json", "r") as f:
    one_other = json.load(f)

In [36]:
one_other[0]["conversations"]

[{'from': 'human',
  'value': 'Mary does her grocery shopping on Saturday. She does her shopping only at a specific store where she is allowed a credit of $100, which must be paid in full before her next shopping trip. That week she spent the full credit limit and paid $15 of it on Tuesday and $23 of it on Thursday. How much credit will Mary need to pay before her next shopping trip?The problem involves finding out how much credit Mary still needs to pay before her next shopping trip. We know that she spent her full credit limit of $100, but has only paid back a portion of it so far.\n<[COT]>'},
 {'from': 'gpt',
  'value': 'Firstly, we need to calculate how much Mary has already paid back by adding up all her payments.</[COT]>'}]

#### 构建hcot-model-llm部分的训练数据

In [5]:
import json
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/h_cot_direct_conv_0126.json",
    "r",
) as f:
    h_cot_math_direct = json.load(f)

In [23]:
import re


def remove_thoughts(text):
    """
    Removes all occurrences of <thought>xxx</thought> from the text.

    Args:
    text (str): The input string containing <thought>xxx</thought> tags.

    Returns:
    str: The modified string with <thought>xxx</thought> tags removed.
    """
    # Pattern to match <thought>xxx</thought> and optional preceding and following newlines
    pattern = r"(?<=\n)<thought>.*?</thought>(?=\n)"

    # Replace all occurrences with an empty string
    modified_text = re.sub(pattern, "<[COT]>", text, flags=re.DOTALL)

    return modified_text

In [24]:
h_cot_math_hcot_llm_model = []
for data in h_cot_math_direct:
    conv = data["conversations"]
    conv_response = conv[1]["value"]
    conv_response_with_cot = remove_thoughts(conv_response)
    conv[1]["value"] = conv_response_with_cot
    h_cot_math_hcot_llm_model.append(data)

In [26]:
len(h_cot_math_hcot_llm_model)

4589

In [25]:
h_cot_math_hcot_llm_model[0]

{'id': '4866e6aa-1274-4c53-a6c7-11df6f31deab',
 '_id': 1062250,
 'created_at': '2024-01-24 15:06:00',
 'updated_at': '2024-01-24 15:07:22',
 'status': 3,
 'channel': 'default',
 'response': "[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem is asking us to factorize a quadratic expression, $t^2 - 121$. This can be done by recognizing that it's a difference of squares, which can be factored into two binomial expressions.\\n<thought>The given expression $t^2 - 121$ is in the form of $a^2 - b^2$, where $a = t$ and $b = 11$. The difference of squares formula states that any expression in this form can be factored as $(a-b)(a+b)$.</thought>\\n<equation>t^2 - 121 = (t-11)(t+

In [27]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/hcot_llm_data_0126.json",
    "w",
) as f:
    json.dump(h_cot_math_hcot_llm_model, f, ensure_ascii=False)

In [29]:
h_cot_math_hcot_llm_model[0]

{'id': '4866e6aa-1274-4c53-a6c7-11df6f31deab',
 '_id': 1062250,
 'created_at': '2024-01-24 15:06:00',
 'updated_at': '2024-01-24 15:07:22',
 'status': 3,
 'channel': 'default',
 'response': "[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem is asking us to factorize a quadratic expression, $t^2 - 121$. This can be done by recognizing that it's a difference of squares, which can be factored into two binomial expressions.\\n<thought>The given expression $t^2 - 121$ is in the form of $a^2 - b^2$, where $a = t$ and $b = 11$. The difference of squares formula states that any expression in this form can be factored as $(a-b)(a+b)$.</thought>\\n<equation>t^2 - 121 = (t-11)(t+

## 测试数据聚合并重新format一下

In [2]:
import json

In [8]:
def read_jsonl(jsonl_path):
    with open(jsonl_path, "r") as f:
        jsonl_lines = f.readlines()
    json_files = []
    for line in jsonl_lines:
        json_files.append(json.loads(line.strip("\n")))
    return json_files

In [9]:
math_test_0 = read_jsonl(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/algebra/test.jsonl"
)
math_test_1 = read_jsonl(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/counting_and_probability/test.jsonl"
)
math_test_2 = read_jsonl(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/number_theory/test.jsonl"
)
math_test_3 = read_jsonl(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/prealgebra/test.jsonl"
)

In [10]:
math_test_all = math_test_0 + math_test_1 + math_test_2 + math_test_3

In [12]:
for one_math_test_json in math_test_all:
    one_math_test_json['instruction'] = ""

In [14]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/MATH/reformat_test.json",
    "w",
) as f:
    json.dump(math_test_all,f, ensure_ascii=False, indent=4)